In [55]:
import pandas as pd
import numpy as np
import seaborn as sns

In [56]:
train_df = pd.read_csv('/home/howt51/work/vscode_micromamba/kaggle/data/playground-series-s3e18/train.csv')
test_df = pd.read_csv('/home/howt51/work/vscode_micromamba/kaggle/data/playground-series-s3e18/test.csv')

In [57]:
set(test_df.columns).symmetric_difference(set(train_X.columns))

{'MolWt', 'NumAromaticRings', 'NumRotatableBonds'}

In [58]:
def process(df):
    df['MolWt'] = df['HeavyAtomMolWt'] + df['ExactMolWt']
    df['NumRotatableBonds'] = df['Chi3v'] + df['Chi4n']
    df['NumAromaticRings'] = df['BertzCT'] - df['Chi1']
    if 'EC3' in df.columns:
        df.drop(['EC3', 'EC4', 'EC5', 'EC6'], axis=1,inplace=True)
    # df.drop(['fr_COO2', 'FpDensityMorgan2', 'FpDensityMorgan3', 'Chi1v', 'Chi1n', 'Chi2n', 'Chi2v', 'Chi3v', 'Chi4n'], axis=1,inplace=True)
    return df
train_df = process(train_df)
test_df = process(test_df)

In [59]:
train_X.columns

Index(['id', 'BertzCT', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3v',
       'Chi4n', 'EState_VSA1', 'EState_VSA2', 'ExactMolWt', 'FpDensityMorgan1',
       'FpDensityMorgan2', 'FpDensityMorgan3', 'HallKierAlpha',
       'HeavyAtomMolWt', 'Kappa3', 'MaxAbsEStateIndex', 'MinEStateIndex',
       'NumHeteroatoms', 'PEOE_VSA10', 'PEOE_VSA14', 'PEOE_VSA6', 'PEOE_VSA7',
       'PEOE_VSA8', 'SMR_VSA10', 'SMR_VSA5', 'SlogP_VSA3', 'VSA_EState9',
       'fr_COO', 'fr_COO2', 'MolWt', 'NumRotatableBonds', 'NumAromaticRings'],
      dtype='object')

In [60]:
train_df_ = train_df.copy()

# y = train_df_[['EC1', 'EC2']]
train_y1 = train_df_['EC1']
train_y2 = train_df_['EC2']
train_X = train_df_.drop(['EC1', 'EC2'], axis=1)

# pycaret data setup

In [61]:
data1 = pd.concat([train_X, train_y1], axis=1)
data2 = pd.concat([train_X, train_y2], axis=1)

In [62]:
data2.columns

Index(['id', 'BertzCT', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3v',
       'Chi4n', 'EState_VSA1', 'EState_VSA2', 'ExactMolWt', 'FpDensityMorgan1',
       'FpDensityMorgan2', 'FpDensityMorgan3', 'HallKierAlpha',
       'HeavyAtomMolWt', 'Kappa3', 'MaxAbsEStateIndex', 'MinEStateIndex',
       'NumHeteroatoms', 'PEOE_VSA10', 'PEOE_VSA14', 'PEOE_VSA6', 'PEOE_VSA7',
       'PEOE_VSA8', 'SMR_VSA10', 'SMR_VSA5', 'SlogP_VSA3', 'VSA_EState9',
       'fr_COO', 'fr_COO2', 'MolWt', 'NumRotatableBonds', 'NumAromaticRings',
       'EC2'],
      dtype='object')

In [63]:

# init setup
from pycaret.classification import ClassificationExperiment
s = ClassificationExperiment()
s.setup(data1, target = 'EC1', session_id = 123, use_gpu=False)

# model training and selection
best = s.compare_models()

# evaluate trained model
s.evaluate_model(best)

[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recomp

,Description,Value
0,Session id,123
1,Target,EC1
2,Target type,Binary
3,Original data shape,"(14838, 36)"
4,Transformed data shape,"(14838, 36)"
5,Transformed train set shape,"(10386, 36)"
6,Transformed test set shape,"(4452, 36)"
7,Numeric features,35
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7060,0.7054,0.8688,0.7376,0.7978,0.2719,0.2848,0.1010
catboost,CatBoost Classifier,0.7026,0.7043,0.8705,0.7337,0.7963,0.2601,0.2738,34.2550
et,Extra Trees Classifier,0.7024,0.6947,0.8689,0.7342,0.7958,0.2608,0.2741,0.5520
lightgbm,Light Gradient Boosting Machine,0.7016,0.6948,0.8682,0.7338,0.7953,0.2589,0.2720,0.0720
rf,Random Forest Classifier,0.7008,0.6948,0.8683,0.7330,0.7949,0.2565,0.2696,0.2930
ada,Ada Boost Classifier,0.6977,0.6926,0.8446,0.7397,0.7886,0.2661,0.2736,0.1050
lda,Linear Discriminant Analysis,0.6944,0.6717,0.8940,0.7177,0.7962,0.2137,0.2357,0.1350
ridge,Ridge Classifier,0.6915,0.0000,0.9019,0.7125,0.7961,0.1969,0.2221,0.0720
lr,Logistic Regression,0.6909,0.6551,0.9116,0.7090,0.7976,0.1853,0.2134,0.0900
xgboost,Extreme Gradient Boosting,0.6774,0.6686,0.8366,0.7235,0.7759,0.2104,0.2180,0.7810


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [64]:

# init setup
from pycaret.classification import ClassificationExperiment
s = ClassificationExperiment()
s.setup(data2, target = 'EC2', session_id = 123, use_gpu=False)

# model training and selection
best = s.compare_models()

# evaluate trained model
s.evaluate_model(best)

,Description,Value
0,Session id,123
1,Target,EC2
2,Target type,Binary
3,Original data shape,"(14838, 36)"
4,Transformed data shape,"(14838, 36)"
5,Transformed train set shape,"(10386, 36)"
6,Transformed test set shape,"(4452, 36)"
7,Numeric features,35
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.7991,0.0000,0.9999,0.7991,0.8883,0.0013,0.0108,0.0290
dummy,Dummy Classifier,0.7990,0.5000,1.0000,0.7990,0.8882,0.0000,0.0000,0.0190
gbc,Gradient Boosting Classifier,0.7984,0.5793,0.9981,0.7994,0.8878,0.0045,0.0229,0.0260
catboost,CatBoost Classifier,0.7982,0.5619,0.9987,0.7990,0.8877,0.0002,0.0012,0.1080
lda,Linear Discriminant Analysis,0.7981,0.5707,0.9987,0.7989,0.8877,-0.0006,-0.0045,0.0260
lr,Logistic Regression,0.7979,0.5479,0.9977,0.7992,0.8875,0.0024,0.0051,0.5360
ada,Ada Boost Classifier,0.7979,0.5751,0.9973,0.7994,0.8875,0.0041,0.0142,0.0370
et,Extra Trees Classifier,0.7973,0.5636,0.9946,0.8003,0.8869,0.0125,0.0358,0.4240
rf,Random Forest Classifier,0.7970,0.5625,0.9963,0.7992,0.8869,0.0024,0.0101,0.1080
lightgbm,Light Gradient Boosting Machine,0.7956,0.5665,0.9935,0.7994,0.8859,0.0040,0.0110,0.0360


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…